# Opdracht schilderijen

In [ ]:
from tensorflow import keras 
from keras import layers
import os
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def create_data_with_labels(dataset_dir):
    """
    Labelt de data (0 = Mondriaan, 1 = Picasso, 2 = Rubens, ...)
    Print welke files corrupted zijn. (bv. data\Picasso\\150.jpg --> FAILED)

    Parameters:
        dataset_dir: A string containing the path to a directory containing
            subdirectories to different classes.
    Returns:
        de data met de labels
    """
    image_paths_per_label = collect_paths_to_files(dataset_dir)

    images = []
    labels = []
    for label, image_paths in image_paths_per_label.items():
        for image_path in image_paths:

            # print(str(image_path))

            img = cv2.imread(str(image_path))

            if(img is not None):
                # print(f"{i} {str(image_path)} --> succes")
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)
                
                # print(label)
                labels.append(label)       
                      
            else:
                print(f"{str(image_path)} --> FAILED")
                
    data = np.array([preprocess_image(image.astype(np.float32))
                for image in images])
    
    labels = np.array(labels)
        
    return data, labels

def collect_paths_to_files(dataset_dir):
    """Returns a dict with labels for each subdirectory of the given directory
    as keys and lists of the subdirectory's contents as values.

    Parameters:
        dataset_dir: A string containing the path to a directory containing
            subdirectories to different classes.
    Returns:
        image_paths_per_label: A dict with labels as keys and lists of file
        paths as values.
    """
    dataset_dir = Path(dataset_dir)
    painter_dirs = [f for f in sorted(os.listdir(dataset_dir)) if not f.startswith('.')]
    image_paths_per_label = {
        label: [
            dataset_dir / painter_dir / '{0}'.format(f)
            for f in os.listdir(dataset_dir / painter_dir) if not f.startswith('.')
        ]
        for label, painter_dir in enumerate(painter_dirs)
    }
    return image_paths_per_label

def preprocess_image(image):
    """Returns a preprocessed image.

    Parameters:
        image: A RGB image with pixel values in range [0, 255].
    Returns
        image: The preprocessed image.
    """

    image = cv2.resize(image, (180, 180))
    image = image / 255.
    
    return image


In [ ]:
(train_data, train_labels) = create_data_with_labels("dataset/train/")
(val_data, val_labels) = create_data_with_labels("dataset/validation")
(test_data, test_labels) = create_data_with_labels("dataset/test")

## Model opbouwen

In [ ]:
input_layer = keras.Input(shape=(180, 180, 3)) 

x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(input_layer)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(3, activation="softmax")(x)  # 4 because we have 4 classes (0 through 3)

model = keras.Model(inputs=input_layer, outputs=outputs)

# Return the compiled model
model.compile(loss="sparse_categorical_crossentropy",
  # categorical crossentropy because we're dealing with multiclass, single label classification
  optimizer=keras.optimizers.RMSprop(),
  metrics=["accuracy"],  # =0.00001
  )

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath="convnet_from_scratch.keras",  # file waarin model wordt opgeslagen
    save_best_only=True,  # beste model wordt opgeslagen obv validation loss!
    monitor="val_loss")
]

history = model.fit(
    train_data, train_labels,
    epochs=30,
    validation_data=(val_data, val_labels),
    callbacks=callbacks
)

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2)

fig.set_size_inches(12, 3)

ax1.plot(epochs, accuracy, "bo", label="Training accuracy")
ax1.plot(epochs, val_accuracy, "b", label="Validation accuracy")
ax1.set_title("Training and validation accuracy")
ax1.legend()
# plt.figure()

ax2.plot(epochs, loss, "bo", label="Training loss")
ax2.plot(epochs, val_loss, "b", label="Validation loss")
ax2.set_title("Training and validation loss")
ax2.legend()
plt.show()

In [ ]:
model.summary()